# Introduction to the Data

In [1]:
import sqlite3
conn = sqlite3.connect("nominations.db")
schema = conn.execute("pragma table_info(nominations);").fetchall()
first_ten = conn.execute("select * from nominations limit 10;").fetchall()

for r in schema:
    print(r)
    
for r in first_ten:
    print(r)

(0, 'Year', 'INTEGER', 0, None, 0)
(1, 'Category', 'TEXT', 0, None, 0)
(2, 'Nominee', 'TEXT', 0, None, 0)
(3, 'Won', 'INTEGER', 0, None, 0)
(4, 'Movie', 'TEXT', 0, None, 0)
(5, 'Character', 'TEXT', 0, None, 0)
(2010, 'Actor -- Leading Role', 'Javier Bardem', 0, 'Biutiful', 'Uxbal')
(2010, 'Actor -- Leading Role', 'Jeff Bridges', 0, 'True Grit', 'Rooster Cogburn')
(2010, 'Actor -- Leading Role', 'Jesse Eisenberg', 0, 'The Social Network', 'Mark Zuckerberg')
(2010, 'Actor -- Leading Role', 'Colin Firth', 1, "The King's Speech", 'King George VI')
(2010, 'Actor -- Leading Role', 'James Franco', 0, '127 Hours', 'Aron Ralston')
(2010, 'Actor -- Supporting Role', 'Christian Bale', 1, 'The Fighter', 'Dicky Eklund')
(2010, 'Actor -- Supporting Role', 'John Hawkes', 0, "Winter's Bone", 'Teardrop')
(2010, 'Actor -- Supporting Role', 'Jeremy Renner', 0, 'The Town', 'James Coughlin')
(2010, 'Actor -- Supporting Role', 'Mark Ruffalo', 0, 'The Kids Are All Right', 'Paul')
(2010, 'Actor -- Supporting 

# Creating the ceremonies table

In [2]:
#conn.execute('drop table ceremonies;')
create_ceremonies = '''create table ceremonies (
                    id integer primary key,
                    Year integer,
                    Host text                    
            );'''
conn.execute(create_ceremonies)

years_hosts = [(2010, "Steve Martin"),
               (2009, "Hugh Jackman"),
               (2008, "Jon Stewart"),
               (2007, "Ellen DeGeneres"),
               (2006, "Jon Stewart"),
               (2005, "Chris Rock"),
               (2004, "Billy Crystal"),
               (2003, "Steve Martin"),
               (2002, "Whoopi Goldberg"),
               (2001, "Steve Martin"),
               (2000, "Billy Crystal"),
            ]
query = 'insert into ceremonies (Year, Host) values (?,?);'
conn.executemany(query, years_hosts)

print(conn.execute('select * from ceremonies limit 10;').fetchall())
print(conn.execute('pragma table_info(ceremonies)').fetchall())

[(1, 2010, 'Steve Martin'), (2, 2009, 'Hugh Jackman'), (3, 2008, 'Jon Stewart'), (4, 2007, 'Ellen DeGeneres'), (5, 2006, 'Jon Stewart'), (6, 2005, 'Chris Rock'), (7, 2004, 'Billy Crystal'), (8, 2003, 'Steve Martin'), (9, 2002, 'Whoopi Goldberg'), (10, 2001, 'Steve Martin')]
[(0, 'id', 'integer', 0, None, 1), (1, 'Year', 'integer', 0, None, 0), (2, 'Host', 'text', 0, None, 0)]


# Foreign Key Constrains

In [3]:
conn.execute('PRAGMA foreign_keys = ON;')

# Setting up one-to-many

In [4]:
create_nom2 = '''create table nominations_two (
                    id integer primary key,
                    category text,
                    nominee text,
                    movie text,
                    character text,
                    won integer,
                    ceremony_id integer,
                    foreign key(ceremony_id) references ceremonies(id));
'''
conn.execute(create_nom2)

In [5]:
join_q = '''
    SELECT ceremonies.id as ceremony_id, nominations.category as category,
    nominations.nominee as nominee, nominations.movie as movie,
    nominations.character as character, nominations.won as won
    FROM nominations
    INNER JOIN ceremonies ON nominations.year == ceremonies.year;
'''
joined_nominations = conn.execute(join_q).fetchall()
print(joined_nominations)

[(1, 'Actor -- Leading Role', 'Javier Bardem', 'Biutiful', 'Uxbal', 0), (1, 'Actor -- Leading Role', 'Jeff Bridges', 'True Grit', 'Rooster Cogburn', 0), (1, 'Actor -- Leading Role', 'Jesse Eisenberg', 'The Social Network', 'Mark Zuckerberg', 0), (1, 'Actor -- Leading Role', 'Colin Firth', "The King's Speech", 'King George VI', 1), (1, 'Actor -- Leading Role', 'James Franco', '127 Hours', 'Aron Ralston', 0), (1, 'Actor -- Supporting Role', 'Christian Bale', 'The Fighter', 'Dicky Eklund', 1), (1, 'Actor -- Supporting Role', 'John Hawkes', "Winter's Bone", 'Teardrop', 0), (1, 'Actor -- Supporting Role', 'Jeremy Renner', 'The Town', 'James Coughlin', 0), (1, 'Actor -- Supporting Role', 'Mark Ruffalo', 'The Kids Are All Right', 'Paul', 0), (1, 'Actor -- Supporting Role', 'Geoffrey Rush', "The King's Speech", 'Lionel Logue', 0), (1, 'Actress -- Leading Role', 'Annette Bening', 'The Kids Are All Right', 'Nic', 0), (1, 'Actress -- Leading Role', 'Nicole Kidman', 'Rabbit Hole', 'Becca', 0), (1,

In [6]:
insert_nominations_two = '''insert into nominations_two (ceremony_id, category, nominee, movie, character, won) 
values (?,?,?,?,?,?);
'''
conn.executemany(insert_nominations_two, joined_nominations)
print(conn.execute('select * from nominations_two limit 5').fetchall())

[(1, 'Actor -- Leading Role', 'Javier Bardem', 'Biutiful', 'Uxbal', 0, 1), (2, 'Actor -- Leading Role', 'Jeff Bridges', 'True Grit', 'Rooster Cogburn', 0, 1), (3, 'Actor -- Leading Role', 'Jesse Eisenberg', 'The Social Network', 'Mark Zuckerberg', 0, 1), (4, 'Actor -- Leading Role', 'Colin Firth', "The King's Speech", 'King George VI', 1, 1), (5, 'Actor -- Leading Role', 'James Franco', '127 Hours', 'Aron Ralston', 0, 1)]


# Deleting and Renaming Tables

In [7]:
conn.execute('drop table nominations;')
conn.execute('alter table nominations_two rename to nominations;')

# Creating a join table

In [8]:
#conn.execute('drop table movies;')
#conn.execute('drop table actors;')
#conn.execute('drop table movies_actors;')

create_movies = 'create table movies (id integer primary key, movie text);'
create_actors = 'create table actors (id integer primary key, actor text);'
create_movies_actors = ''' create table movies_actors(
    id integer primary key, 
    movie_id integer references movies(id),
    actor_id integer references actors(id));
'''

conn.execute(create_movies)
conn.execute(create_actors)
conn.execute(create_movies_actors)